#### Utility code to convert from CSV to feather for faster load times

In [ ]:
import pandas as pd
import feather

#################### READ CSV #########################
# df = pd.read_csv('./all/train.csv', parse_dates=['pickup_datetime'])
################## WRITE FEATHER ######################
# df.to_feather('./all/train.feather')
################## READ FEATHER #######################
df = pd.read_feather('./all/train.feather')

# Describe the original dataset

In [ ]:
df.describe()

### Instantly recognizable outliers:
* **passenger_count**
    * Passenger counts of 0
    * Infeasible passenger counts (e.g. 200). Taxis can legally hold up to 4-6 passengers (http://www.nyc.gov/html/tlc/html/faq/faq_pass.shtml)
* **fare_amount**
    * Negative fare amounts
    * Extremely high fare amounts (thousands of dollars)
* **coordinates**
    * Invalid coordinates (out of defined range)
    * Coordinates outside of NYC

***
# Data cleaning functions:
#### Check for invalid coordinates (outside of NYC range), determined with https://www.mapdevelopers.com/geocode_bounding_box.php

In [1]:
def valid_coordinates(lat_list, lon_list):
    for i in lat_list:
        if i < 40.477399 or i > 40.917577:
            return False
    for i in lon_list:
        if i < -74.259090 or i > -73.700272:
            return False
    return True

def clean_coordinates(df):
    df.query('~(pickup_latitude < 40.477399 or pickup_latitude > 40.917577) &\
              ~(dropoff_latitude < 40.477399 or dropoff_latitude > 40.917577) &\
              ~(pickup_longitude < -74.259090 or pickup_longitude > -73.700272) &\
              ~(dropoff_longitude < -74.259090 or dropoff_longitude > -73.700272)', inplace=True)

#### Remove invalid passenger counts, fares. Upper bound to both. Also drop missing values and key column.

In [2]:
def clean_pfdk(df):
    df.drop(columns=['key'], inplace=True)
    df.dropna(inplace=True)
    df.query('passenger_count > 0 &\
              passenger_count <= 6 &\
              fare_amount > 0 &\
              fare_amount <= 100 ', inplace=True)

***
# Feature Engineering functions:
***
#### Calculate Distance (and drop rows with distance = 0 if training)

In [3]:
def euclidean_distance(x1, y1, x2, y2):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5

def manhattan_distance(x1, y1, x2, y2):
    return abs(x2-x1) + abs(y2-y1)
    
def add_euclidean_distance(df, training=True):
    df['euclidean_distance'] = euclidean_distance(df['pickup_latitude'], df['pickup_longitude'], df['dropoff_latitude'], df['dropoff_longitude'])
    if training:
        df.query('euclidean_distance > 0', inplace=True)
        df.reset_index(drop=True, inplace=True)
    
def add_manhattan_distance(df, training=True):
    df['manhattan_distance'] = manhattan_distance(df['pickup_latitude'], df['pickup_longitude'], df['dropoff_latitude'], df['dropoff_longitude'])
    if training:
        df.query('manhattan_distance > 0', inplace=True)
        df.reset_index(drop=True, inplace=True)

#### Extract Time Values

In [4]:
def add_times(df):
    df['year']  = df['pickup_datetime'].dt.year
    df['month'] = df['pickup_datetime'].dt.month
    df['day']   = df['pickup_datetime'].dt.day
    df['hour']  = df['pickup_datetime'].dt.hour + (df['pickup_datetime'].dt.minute/60)
    df.drop(columns=['pickup_datetime'], inplace=True)

#### External data: Monthly Gas Prices acquired from https://data.bls.gov/timeseries/APU000074714

In [5]:
def add_gas_prices(df):
    gas_df = pd.read_csv('./all/gas_prices.csv')
    gas_dict = {}
    for row in gas_df.itertuples():
        year = row[1]
        for i in range(2, len(row)):
            gas_dict['{0}-{1}'.format(year, i-1)] = row[i]
    df['year-month']  = df['year'].astype(str) + '-' + df['month'].astype(str)
    df['gas'] = df['year-month'].map(gas_dict)
    df.drop(columns=['year-month'], inplace=True)

#### Preprocess pipeline function

In [6]:
def preprocess(df, training=True):
    if training:
        clean_pfdk(df)
        clean_coordinates(df)
        df.reset_index(drop=True, inplace=True)
    add_euclidean_distance(df, training)
    add_manhattan_distance(df, training)
    add_times(df)
    add_gas_prices(df)

#### Utility code to preprocess training data and save as feather

In [35]:
import pandas as pd
import feather

############## PREPROCESS TRAINING DATA $##############
# preprocess(df, training=True)
############## WRITE PREPROCESSED FEATHER #############
# df.to_feather('./all/preprocessed_train.feather')
######### READ PREPROCESSED FEATHER ###################
df = pd.read_feather('./all/preprocessed_train.feather')

***
# Visualization:
#### Scatter plot (Euclidean Distance, Fare Amount)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

corr = df['euclidean_distance'].corr(df['fare_amount'])
plt.figure(figsize=(10,10))
plt.scatter(df['euclidean_distance'], df['fare_amount'], alpha=0.2)
plt.title('ScatterPlot (Euclidean Distance, Fare Amount)\nCorrelation = {0}'.format(corr), fontsize=20)
plt.xlabel('Euclidean Distance', fontsize=18)
plt.ylabel('Fare Amount', fontsize=18)
plt.show()

Observations:
* Very strong correlation (0.87)
* There are three groups of fixed fares over any distance. This makes sense because there are usually fixed fares to JFK, Newark, and LaGuardia airports. 

#### ScatterPlot (Time of Day, Distance Traveled)

In [ ]:
corr = df['hour'].corr(df['euclidean_distance'])
plt.figure(figsize=(10,10))
plt.scatter(df['hour'], df['euclidean_distance'], alpha=0.05)
plt.title('ScatterPlot (Time of Day, Distance Traveled)\nCorrelation = {0}'.format(corr), fontsize=20)
plt.xlabel('Time of day', fontsize=18)
plt.ylabel('Euclidean distance', fontsize=18)
plt.xlim(0,24)
plt.xticks(range(0,25))
plt.grid(True)
plt.show()

Observations:
* Seems to be very low/negligible correlation (-0.03)
* Naturally, less people travel in the early morning

#### ScatterPlot (Time of Day, Fare Amount)

In [ ]:
corr = df['hour'].corr(df['fare_amount'])
plt.figure(figsize=(10,10))
plt.scatter(df['hour'], df['fare_amount'], alpha=0.05)
plt.title('ScatterPlot (Time of Day, Fare Amount)\nCorrelation = {0}'.format(corr), fontsize=20)
plt.xlabel('Time of day', fontsize=18)
plt.ylabel('Fare Amount', fontsize=18)
plt.xlim(0,24)
plt.xticks(range(0,25))
plt.grid(True)
plt.show()

Observations:
* Correlation also very weak (-0.018)
* Three groups of fixed fares also visible here

#### Correlation Heatmap

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr_matrix, cmap='BuGn', annot=True, vmin=-1, vmax=1)
plt.show()

Observations:
* Euclidean distance has the highest correlation with fare amount
* Latidude is negatively correlated with fare
* Longitude is positively correlated with fare
* Year has a correlation of 0.12 with fare. Makes sense since costs slowly rise with inflation.
* Gas prices and year have significant correlation (0.4)

#### Fares by Destination

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df['dropoff_longitude'], df['dropoff_latitude'], c=df['fare_amount'], cmap=plt.get_cmap('jet'), alpha=0.05)
colorbar = plt.colorbar()
colorbar.ax.set_ylabel('Fare Amount', fontsize=20)
plt.title('Fares by Destination'.format(corr), fontsize = 20)
plt.xlabel('Longitude', fontsize=18)
plt.ylabel('Latitude', fontsize=18)
plt.show()

***   
# Train/Test split wrapper function
Can specify number of rows to train on and test subset size.

In [19]:
# SPLIT TRAIN/TEST DATA
from sklearn.model_selection import train_test_split

def get_tt_split(df, nrows, test_size=0.2):
    X = df.head(nrows)[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'year', 'month', 'day', 'hour', 'manhattan_distance', 'euclidean_distance', 'passenger_count', 'gas']]
    y = df.head(nrows)['fare_amount']
    return train_test_split(X, y, test_size=test_size, random_state=3)

***
# Model: Linear Regression

In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def train_lr(df):
    X_train, X_test, y_train, y_test = get_tt_split(df, nrows=len(df), test_size=0.005)
    
    model = LinearRegression()
    print('Training model: {0}...'.format('lr'))
    model.fit(X_train, y_train)

    test_predictions = model.predict(X_test)
    print('Mean Squared Error (Training):', mean_squared_error(y_test, test_predictions))
    print('\nCOEFFICIENTS:')
    for i in list(zip(X_test.columns, model.coef_)):
        print('{0}: {1}'.format(i[0], i[1]))
    print('==============================')
    print('\nINTERCEPT:', model.intercept_)
    
    return model

#### Training Results:

In [55]:
%%time
train_lr(df)

Training model: lr...


MemoryError: 

Observations:
* Most important feature:
* Scaling has no effect on performance

# Model: Decision Tree Ensembles (XGBoost)

In [ ]:
# XGBOOST (DECISION TREE ENSEMBLES)

import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error

def train_xgb(df):
    X_train, X_test, y_train, y_test = get_tt_split(df, nrows=len(df), test_size=0.005)
    
    print('Training model: {0}...'.format('lr'))
    model = xgb.train(params={'eta':0.6}, dtrain=DMatrix(X_train, y_train))

    test_predictions = model.predict(xgb.DMatrix(X_test))
    print('Mean Squared Error (Training):', mean_squared_error(y_test, test_predictions))
    
    return model

#### Training Results:

In [ ]:
%%time
train_xgb(df)

# Model: Neural Network

In [53]:
# NEURAL NETWORK

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.train import RMSPropOptimizer
from tensorflow.nn import relu, softmax
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

def train_nn(df):
    X_train, X_test, y_train, y_test = get_tt_split(df, nrows=100000, test_size=0.2)
    X_train = MinMaxScaler().fit_transform(X_train)
    X_test = MinMaxScaler().fit_transform(X_test)
    
    shape = (len(X_train[0]),)
    learning_rate = 0.01
    model = keras.Sequential([Dense(128, input_shape=shape), Dense(128, activation=relu), Dense(1)])
    model.compile(loss='mse', metrics=['mse'], optimizer=RMSPropOptimizer(learning_rate))
    print('Training model: {0}...'.format('lr'))
    model.fit(X_train, y_train, epochs=300, verbose=0)

    test_predictions = [i[0] for i in model.predict(X_test)]
    print('Mean Squared Error (Training):', mean_squared_error(y_test, test_predictions))
    
    return model

#### Training Results:

In [52]:
%%time
train_nn(df)

Mean Squared Error (Training): 18.902572218543433
Wall time: 23min 10s


Observations:
* Outperforms Linear Regression model with only a small fraction of the data
* Takes significantly longer than other models to run
* Scaling improves performance significantly

# Specify model:

In [ ]:
selected_model = 'nn'

# Preprocess/fit model to real test data:

In [ ]:
real_df = pd.read_csv('./all/test.csv', parse_dates=['pickup_datetime'])
preprocess(real_df, training=False)

real_X = real_df[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'year', 'month', 'day', 'hour', 'manhattan_distance', 'euclidean_distance', 'passenger_count', 'gas']]

if scaled:
    real_X = StandardScaler().fit_transform(real_X)

if selected_model == 'lr':
    model = train_lr(df)
    predictions = model.predict(real_X)
elif selected_model == 'xgb':
    model = train_xgb(df)
    predictions = model.predict(xgb.DMatrix(real_X))
elif selected_model == 'nn':
    model = train_nn(df)
    predictions = [i[0] for i in model.predict(real_X)]

data = list(zip(real_df['key'], predictions))

submission = pd.DataFrame(data, columns=['key', 'fare_amount'])
submission.set_index('key', inplace=True)
submission.to_csv('submission.csv')
print('----------------------------')
print('Predictions written to submissions.csv')

## Kaggle Scores by model (RMSE):
* Linear Regression: 5.64822
* Neural Network: 4.36043
* Decision Tree Ensembles (XGBoost): 3.51330    <---